# Image Thresholding for Condensate Detection

This notebook implements automated thresholding techniques to segment and extract biomolecular condensates from microscopy images.

## Pipeline Overview
1. **Image Loading**: Load images from categorized directories (Homogenous, Aggregate, Condensate)
2. **Histogram Analysis**: Analyze pixel intensity distributions
3. **Threshold Detection**: Apply KDE method to find optimal thresholds
4. **Binary Segmentation**: Convert images to binary masks
5. **Morphological Processing**: Apply erosion+dilation to remove noise while preserving features
6. **Batch Processing**: Process and save all images with optimal threshold

## Method: KDE (Kernel Density Estimation)
Uses statistical analysis to detect histogram tail inflection points, identifying the transition from signal to background.

## Import Dependencies

In [ ]:
# Standard library
import os
from glob import glob
import random

# Scientific computing
import numpy as np
from PIL import Image

# Image processing
import cv2
from scipy.stats import gaussian_kde
from scipy import signal

# Visualization
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

## Data Loading Functions

In [ ]:
def load_images_with_metadata(root_path):
    """
    Load images from specified subfolders and store them with metadata.
    
    Expected directory structure:
        root_path/
            ├── Homogenous/
            ├── Aggregate/
            └── Condensate/
    
    Args:
        root_path (str): Path to the root directory containing category subfolders
        
    Returns:
        list: List of dictionaries with keys: 'image', 'path', 'filename', 'category'
    """
    subfolders = ['Homogenous', 'Aggregate', 'Condensate']
    image_data = []
    image_extensions = ['*.png', '*.jpg', '*.jpeg', '*.tif', '*.tiff']
    
    for subfolder in subfolders:
        folder_path = os.path.join(root_path, subfolder)
        
        if not os.path.exists(folder_path):
            print(f"Warning: {folder_path} does not exist")
            continue
            
        for ext in image_extensions:
            image_paths = glob(os.path.join(folder_path, '**', ext), recursive=True)
            
            for img_path in image_paths:
                try:
                    img = Image.open(img_path)
                    img_array = np.array(img)
                    
                    image_info = {
                        'image': img_array,
                        'path': img_path,
                        'filename': os.path.basename(img_path),
                        'category': subfolder
                    }
                    image_data.append(image_info)
                    
                except Exception as e:
                    print(f"Error loading {img_path}: {str(e)}")
    
    print(f"Successfully loaded {len(image_data)} images")
    return image_data

## Threshold Detection Functions

In [ ]:
def find_tail_kde(non_zero_pixels):
    """
    Find histogram tail start using Kernel Density Estimation and derivative analysis.
    
    This method identifies inflection points in the pixel distribution where the
    curve transitions from the main distribution to the tail (high-intensity regions).
    
    Args:
        non_zero_pixels (array): Array of non-zero pixel values
        
    Returns:
        float: The threshold value where the tail begins
    """
    try:
        # Create KDE of the pixel distribution
        kde = gaussian_kde(non_zero_pixels)
        
        # Evaluate KDE on a fine grid
        x_grid = np.linspace(non_zero_pixels.min(), non_zero_pixels.max(), 1000)
        pdf = kde(x_grid)
        
        # Calculate first derivative of the PDF
        pdf_derivative = np.gradient(pdf, x_grid)
        
        # Find inflection points (where second derivative ≈ 0)
        inflection_candidates = signal.find_peaks(-np.abs(np.gradient(pdf_derivative, x_grid)))[0]
        
        # Look for inflection point in the right half of distribution
        # This identifies the transition to the tail
        median_value = np.median(non_zero_pixels)
        median_idx = np.argmin(np.abs(x_grid - median_value))
        right_inflections = [i for i in inflection_candidates 
                            if i > median_idx and pdf[i] < 0.5 * pdf.max()]
        
        if right_inflections:
            return x_grid[right_inflections[0]]
        else:
            # Fallback: use mean + 1.5 * std
            return np.mean(non_zero_pixels) + 1.5 * np.std(non_zero_pixels)
    except Exception as e:
        print(f"KDE tail detection failed: {e}")
        return np.mean(non_zero_pixels) + 1.5 * np.std(non_zero_pixels)

## Morphological Processing Functions

In [ ]:
def apply_morphological_opening(binary_image, kernel_size=3):
    """
    Apply morphological opening (erosion followed by dilation) to clean binary image.
    
    This operation:
    1. Removes small noise particles (erosion step)
    2. Restores shape of remaining objects (dilation step)
    
    Args:
        binary_image (ndarray): Binary image (0 or 255 values)
        kernel_size (int): Size of the square kernel for morphological operations
        
    Returns:
        ndarray: Cleaned binary image after opening
    """
    # Ensure binary values (0 or 255)
    if np.max(binary_image) == 1:
        binary_image = binary_image * 255
    
    # Create square kernel
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    
    # Apply erosion followed by dilation
    eroded = cv2.erode(binary_image, kernel, iterations=1)
    opened = cv2.dilate(eroded, kernel, iterations=1)
    
    return opened

## Visualization Functions

In [ ]:
def display_processing_pipeline(image_data, n_samples=5, kernel_size=3):
    """
    Display the complete processing pipeline for random image samples.
    
    Shows: Original → Histogram+Threshold → Binary → Morphologically Processed
    
    Args:
        image_data (list): List of image dictionaries
        n_samples (int): Number of random samples to display
        kernel_size (int): Size of morphological kernel
    """
    samples = random.sample(image_data, min(n_samples, len(image_data)))
    
    for img_info in samples:
        img = img_info['image'].astype(np.float32)
        
        # Skip empty images
        non_zero_pixels = img[img > 0]
        if len(non_zero_pixels) == 0:
            continue
        
        # Calculate KDE threshold
        threshold = find_tail_kde(non_zero_pixels)
        
        # Apply threshold
        binary_img = (img > threshold).astype(np.uint8) * 255
        
        # Apply morphological opening
        processed_img = apply_morphological_opening(binary_img, kernel_size)
        
        # Create visualization
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle(f"{img_info['filename'][:20]}... ({img_info['category']})", fontsize=16)
        
        # Original image
        axes[0, 0].imshow(img, cmap='gray')
        axes[0, 0].set_title("Original Image")
        axes[0, 0].axis('off')
        
        # Histogram with threshold
        axes[0, 1].hist(non_zero_pixels, bins=50, alpha=0.7, color='blue')
        axes[0, 1].axvline(threshold, color='red', linestyle='--', 
                         label=f"KDE threshold: {threshold:.1f}")
        axes[0, 1].set_title("Pixel Histogram")
        axes[0, 1].set_xlabel("Pixel Value")
        axes[0, 1].set_ylabel("Frequency")
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # Binary image
        axes[1, 0].imshow(binary_img, cmap='gray')
        axes[1, 0].set_title(f"Binary (threshold={threshold:.1f})")
        axes[1, 0].axis('off')
        
        # Morphologically processed
        axes[1, 1].imshow(processed_img, cmap='gray')
        axes[1, 1].set_title(f"After Opening ({kernel_size}×{kernel_size} kernel)")
        axes[1, 1].axis('off')
        
        plt.tight_layout()
        plt.show()

## Batch Processing Function

In [ ]:
def process_and_save_images(image_data, output_root_path, kernel_size=3):
    """
    Process all images using KDE thresholding + morphological opening and save results.
    
    Args:
        image_data (list): List of image dictionaries
        output_root_path (str): Root directory for saving processed images
        kernel_size (int): Size of morphological kernel
    """
    # Create output directories
    for category in ['Homogenous', 'Condensate', 'Aggregate']:
        os.makedirs(os.path.join(output_root_path, category), exist_ok=True)
    
    category_counts = {'Homogenous': 0, 'Condensate': 0, 'Aggregate': 0}
    
    for i, img_info in enumerate(image_data):
        category = img_info['category']
        filename = img_info['filename']
        target_path = os.path.join(output_root_path, category, filename)
        
        img = img_info['image'].astype(np.float32)
        
        # Extract non-zero pixels
        non_zero_pixels = img[img > 0]
        if len(non_zero_pixels) == 0:
            print(f"Skipping empty image: {filename}")
            continue
        
        try:
            # Calculate KDE threshold
            kde_threshold = find_tail_kde(non_zero_pixels)
            
            # Apply thresholding
            binary_img = (img > kde_threshold).astype(np.uint8) * 255
            
            # Apply morphological opening
            processed_img = apply_morphological_opening(binary_img, kernel_size)
            
            # Save processed image
            Image.fromarray(processed_img).save(target_path)
            category_counts[category] += 1
            
            if (i + 1) % 10 == 0:
                print(f"Processed {i + 1}/{len(image_data)} images...")
                
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
    
    # Print summary
    print("\nProcessing complete!")
    print(f"Total: {sum(category_counts.values())} images")
    for category, count in category_counts.items():
        print(f"{category}: {count} images")
    print(f"Saved to: {output_root_path}")

## Load Dataset

Load images from the dataset directory containing three categories.

In [ ]:
# Configure dataset path
root_path = '/Users/klavs/Desktop/PhD/Tanu/image-thresholding/data/3'  # Update this path

# Load images
images = load_images_with_metadata(root_path)
print(f"Loaded {len(images)} total images")

## Visualize Processing Pipeline

Display the complete processing pipeline on random samples.

In [ ]:
# Display processing pipeline on 5 random samples
display_processing_pipeline(images, n_samples=5, kernel_size=3)

## Batch Process All Images

Apply KDE thresholding + morphological opening to all images and save results.

In [ ]:
# Configure output path
output_path = '/Users/klavs/Desktop/PhD/Tanu/image-thresholding/data/3'  # Update this path

# Process and save all images
process_and_save_images(images, output_path, kernel_size=3)